###### Name: Deepak Vadithala
###### Course: MSc Data Science
###### Project Name: MOOC Recommender System

##### Notes:
This notebook contains the analysis of the **word2vec** model. This model uses google trained model. 
Mutiple variables **(Role and Skill Scores)** are used to predict the course category. **Role and skill** are combined together. Removed noise from the course description field and kept only the keywords which are important.
Skill Score is calculated using the similarity between the skills from LinkedIn compared with the course description with keywords from Coursera.


*Model Source Code Path: /mooc-recommender/Model/Cosine_Distance.py*

*Github Repo: https://github.com/iamdv/mooc-recommender*

In [1]:
# **************************** IMPORTANT ****************************
'''
This cell configuration settings for the Notebook. 
You can run one role at a time to evaluate the performance of the model
Change the variable names to run for multiple roles

In this model:
1. cosine distance is calculated between the skills and the course description 
with the weight of 70%. And each skill has a weighted score based on the 
popularity of the skill. This is derived by endorsements of the respective
skill by other linkedin connections.

2. cosine distance is calcuated between the role and the course name with 
with the weight of 30%.
'''

# *******************************************************************
# For each role a list of category names are grouped. 
# Please don't change these variables

label_DataScientist = ['Data Science','Machine Learning',
                           'Data Analysis', 'Business Intelligence',
                           'Data Mining','Data Visualization', 'Deep Learning']

label_SoftwareDevelopment = ['Software Development','Computer Science',
                           'Programming Languages', 'Software Development',
                           'Web Development','Algorithms and Data Structures', 
                           'Information Technology', 'Mobile Development',
                           'Android Development']


label_DatabaseAdministrator = ['Databases']

label_Cybersecurity = ['Cybersecurity']

label_FinancialAccountant = ['Finance', 'Accounting']

label_MachineLearning = ['Machine Learning', 'Artificial Intelligence',
                         'Data Mining','Data Visualization', 'Deep Learning',
                        'Robotics', 'Statistics & Probability']

label_Musician = ['Music']

label_Dietitian = ['Public Health', 'Nutrition & Wellness', 'Health Care']

label_Psychologist = ['Psychology','Health Care']
            
# *******************************************************************


# *******************************************************************
# Environment and Config Variables. Change these variables as per the requirement.

my_fpath_courses = "../Data/main_coursera.csv"

my_fpath_skills_DataScientist = "../Data//Word2Vec-Google/Word2VecGoogle_DataScientist.csv"

my_fpath_skills_SoftwareDevelopment = "../Data//Word2Vec-Google/Word2VecGoogle_SoftwareDevelopment.csv" 

my_fpath_skills_DatabaseAdministrator = "../Data//Word2Vec-Google/Word2VecGoogle_DatabaseAdministrator.csv"

my_fpath_skills_Cybersecurity = "../Data//Word2Vec-Google/Word2VecGoogle_Cybersecurity.csv"

my_fpath_skills_FinancialAccountant = "../Data//Word2Vec-Google/Word2VecGoogle_FinancialAccountant.csv"

my_fpath_skills_MachineLearning = "../Data//Word2Vec-Google/Word2VecGoogle_MachineLearning.csv"

my_fpath_skills_Musician = "../Data//Word2Vec-Google/Word2VecGoogle_Musician.csv"

my_fpath_skills_Dietitian = "../Data//Word2Vec-Google/Word2VecGoogle_Dietitian.csv"

my_fpath_skills_Psychologist = "../Data//Word2Vec-Google/Word2VecGoogle_Psychologist.csv"

# *******************************************************************


# *******************************************************************
# Weighting Variables. Change them as per the requirement.

my_role_weight = 0

my_skill_weight = 1

# *******************************************************************


In [2]:
# Importing required modules/packages

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, string

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



height has been deprecated.



In [3]:
# Downloading the stopwords like i, me, and, is, the etc.

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/dv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Loading courses and skills data from the CSV files

df_courses = pd.read_csv(my_fpath_courses)

df_DataScientist = pd.read_csv(my_fpath_skills_DataScientist)
df_DataScientist = df_DataScientist.drop('Role', 1)
df_DataScientist.columns = ['Course Id', 'DataScientist_Skill_Score', 'DataScientist_Role_Score', 'DataScientist_Keyword_Score']

df_SoftwareDevelopment = pd.read_csv(my_fpath_skills_SoftwareDevelopment)
df_SoftwareDevelopment = df_SoftwareDevelopment.drop('Role', 1)
df_SoftwareDevelopment.columns = ['Course Id','SoftwareDevelopment_Skill_Score', 'SoftwareDevelopment_Role_Score', 'SoftwareDevelopment_Keyword_Score']

df_DatabaseAdministrator = pd.read_csv(my_fpath_skills_DatabaseAdministrator)
df_DatabaseAdministrator = df_DatabaseAdministrator.drop('Role', 1)
df_DatabaseAdministrator.columns = ['Course Id','DatabaseAdministrator_Skill_Score', 'DatabaseAdministrator_Role_Score', 'DatabaseAdministrator_Keyword_Score']

df_Cybersecurity = pd.read_csv(my_fpath_skills_Cybersecurity)
df_Cybersecurity = df_Cybersecurity.drop('Role', 1)
df_Cybersecurity.columns = ['Course Id','Cybersecurity_Skill_Score', 'Cybersecurity_Role_Score', 'Cybersecurity_Keyword_Score']

df_FinancialAccountant = pd.read_csv(my_fpath_skills_FinancialAccountant)
df_FinancialAccountant = df_FinancialAccountant.drop('Role', 1)
df_FinancialAccountant.columns = ['Course Id','FinancialAccountant_Skill_Score', 'FinancialAccountant_Role_Score', 'FinancialAccountant_Keyword_Score']

df_MachineLearning = pd.read_csv(my_fpath_skills_MachineLearning)
df_MachineLearning = df_MachineLearning.drop('Role', 1)
df_MachineLearning.columns = ['Course Id','MachineLearning_Skill_Score', 'MachineLearning_Role_Score', 'MachineLearning_Keyword_Score']

df_Musician = pd.read_csv(my_fpath_skills_Musician)
df_Musician = df_Musician.drop('Role', 1)
df_Musician.columns = ['Course Id','Musician_Skill_Score', 'Musician_Role_Score', 'Musician_Keyword_Score']

df_Dietitian = pd.read_csv(my_fpath_skills_Dietitian)
df_Dietitian = df_Dietitian.drop('Role', 1)
df_Dietitian.columns = ['Course Id','Dietitian_Skill_Score', 'Dietitian_Role_Score','Dietitian_Keyword_Score']

df_Psychologist = pd.read_csv(my_fpath_skills_Psychologist)
df_Psychologist = df_Psychologist.drop('Role', 1)
df_Psychologist.columns = ['Course Id','Psychologist_Skill_Score', 'Psychologist_Role_Score', 'Psychologist_Keyword_Score']

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [6]:
# Merging the csv files

df_cosdist = df_DataScientist.merge(df_SoftwareDevelopment, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_DatabaseAdministrator, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Cybersecurity, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_FinancialAccountant, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_MachineLearning, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Musician, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Dietitian, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Psychologist, on = 'Course Id', how = 'outer')


NameError: name 'df_DatabaseAdministrator' is not defined

In [7]:
# Exploring data dimensionality, feature names, and feature types.

print(df_courses.shape,"\n")

print(df_cosdist.shape,"\n")

print(df_courses.columns, "\n")

print(df_cosdist.shape,"\n")

print(df_courses.describe(), "\n")

print(df_cosdist.describe(), "\n")


(2213, 19) 

(2213, 6) 

Index(['Unnamed: 0', 'Course Id', 'Course Name', 'Course Description', 'Slug', 'Provider', 'Universities/Institutions', 'Parent Subject', 'Child Subject', 'Category', 'Url', 'Length', 'Language', 'Credential Name', 'Rating', 'Number of Ratings', 'Certificate', 'Workload', 'Course Keywords'], dtype='object') 

(2213, 6) 

        Unnamed: 0    Course Id      Length       Rating  Number of Ratings  Certificate
count  2213.000000  2213.000000  964.000000  2213.000000        2213.000000       2089.0
mean   1106.000000  4816.998192    6.063278     2.352785          10.321735          1.0
std     638.982394  3033.878865    2.724669     2.129134         110.680382          0.0
min       0.000000   303.000000    1.000000     0.000000           0.000000          1.0
25%     553.000000  1829.000000    4.000000     0.000000           0.000000          1.0
50%    1106.000000  4880.000000    6.000000     3.000000           1.000000          1.0
75%    1659.000000  7329.0000

In [8]:
# Quick check to see if the dataframe showing the right results

df_cosdist.head(20)

,Course Id,DataScientist_Skill_Score,DataScientist_Role_Score,DataScientist_Keyword_Score,Skill_Score,Role_Score
0,303,0.353896,0.218317,0.006389,0.329754,0.268359
1,305,0.248907,0.201266,0.060863,0.500153,0.479665
2,306,0.196352,0.111541,0.028367,0.314151,0.105439
3,307,0.325023,0.269903,0.192762,0.300802,0.210988
4,308,0.268324,0.177796,0.096390,0.298195,0.164311
5,309,0.346725,0.279909,0.093165,0.311279,0.330534
6,316,0.330795,0.341219,0.076157,0.253628,0.125416
7,317,0.281024,0.177137,0.065165,0.241946,0.148140
8,318,0.274875,0.276595,0.090191,0.343393,0.263992
9,322,0.337485,0.187397,0.028805,0.274025,0.232131


In [214]:
# Joining two dataframes - Courses and the Cosein Similarity Results based on the 'Course Id' variable. 
# Inner joins: Joins two tables with the common rows. This is a set operateion.

df_courses_score = df_courses.merge(df_cosdist, on ='Course Id', how='inner')

print(df_courses_score.shape,"\n")


(2213, 46) 



In [215]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: DATA SCIENTIST

y_actu_DataScientist         = ''
y_pred_DataScientist         = ''

df_courses_score['DataScientist_Final_Score'] = (df_courses_score['DataScientist_Role_Score'] * my_role_weight) + (df_courses_score['DataScientist_Keyword_Score'] * my_skill_weight)

df_courses_score['DataScientist_Predict'] = (df_courses_score['DataScientist_Final_Score'] >= 0.5)

df_courses_score['DataScientist_Label'] = df_courses_score.Category.isin(label_DataScientist)

y_pred_DataScientist = pd.Series(df_courses_score['DataScientist_Predict'], name='Predicted')

y_actu_DataScientist = pd.Series(df_courses_score['DataScientist_Label'], name='Actual')

df_confusion_DataScientist = pd.crosstab(y_actu_DataScientist, y_pred_DataScientist , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [216]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: SOFTWARE ENGINEER/DEVELOPER

y_actu_SoftwareDevelopment         = ''
y_pred_SoftwareDevelopment         = ''

df_courses_score['SoftwareDevelopment_Final_Score'] = (df_courses_score['SoftwareDevelopment_Role_Score'] * my_role_weight) + (df_courses_score['SoftwareDevelopment_Keyword_Score'] * my_skill_weight)

df_courses_score['SoftwareDevelopment_Predict'] = (df_courses_score['SoftwareDevelopment_Final_Score'] >= 0.5)

df_courses_score['SoftwareDevelopment_Label'] = df_courses_score.Category.isin(label_SoftwareDevelopment)

y_pred_SoftwareDevelopment = pd.Series(df_courses_score['SoftwareDevelopment_Predict'], name='Predicted')

y_actu_SoftwareDevelopment = pd.Series(df_courses_score['SoftwareDevelopment_Label'], name='Actual')

df_confusion_SoftwareDevelopment = pd.crosstab(y_actu_SoftwareDevelopment, y_pred_SoftwareDevelopment , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [217]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: DATABASE DEVELOPER/ADMINISTRATOR

y_actu_DatabaseAdministrator         = ''
y_pred_DatabaseAdministrator         = ''

df_courses_score['DatabaseAdministrator_Final_Score'] = (df_courses_score['DatabaseAdministrator_Role_Score'] * my_role_weight) + (df_courses_score['DatabaseAdministrator_Keyword_Score'] * my_skill_weight)

df_courses_score['DatabaseAdministrator_Predict'] = (df_courses_score['DatabaseAdministrator_Final_Score'] >= 0.5)

df_courses_score['DatabaseAdministrator_Label'] = df_courses_score.Category.isin(label_DatabaseAdministrator)

y_pred_DatabaseAdministrator = pd.Series(df_courses_score['DatabaseAdministrator_Predict'], name='Predicted')

y_actu_DatabaseAdministrator = pd.Series(df_courses_score['DatabaseAdministrator_Label'], name='Actual')

df_confusion_DatabaseAdministrator = pd.crosstab(y_actu_DatabaseAdministrator, y_pred_DatabaseAdministrator , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [218]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: CYBERSECURITY CONSULTANT

y_actu_Cybersecurity         = ''
y_pred_Cybersecurity         = ''

df_courses_score['Cybersecurity_Final_Score'] = (df_courses_score['Cybersecurity_Role_Score'] * my_role_weight) + (df_courses_score['Cybersecurity_Keyword_Score'] * my_skill_weight)

df_courses_score['Cybersecurity_Predict'] = (df_courses_score['Cybersecurity_Final_Score'] >= 0.5)

df_courses_score['Cybersecurity_Label'] = df_courses_score.Category.isin(label_Cybersecurity)

y_pred_Cybersecurity = pd.Series(df_courses_score['Cybersecurity_Predict'], name='Predicted')

y_actu_Cybersecurity = pd.Series(df_courses_score['Cybersecurity_Label'], name='Actual')

df_confusion_Cybersecurity = pd.crosstab(y_actu_Cybersecurity, y_pred_Cybersecurity , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [219]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: FINANCIAL ACCOUNTANT

y_actu_FinancialAccountant         = ''
y_pred_FinancialAccountant         = ''

df_courses_score['FinancialAccountant_Final_Score'] = (df_courses_score['FinancialAccountant_Role_Score'] * my_role_weight) + (df_courses_score['FinancialAccountant_Keyword_Score'] * my_skill_weight)

df_courses_score['FinancialAccountant_Predict'] = (df_courses_score['FinancialAccountant_Final_Score'] >= 0.5)

df_courses_score['FinancialAccountant_Label'] = df_courses_score.Category.isin(label_FinancialAccountant)

y_pred_FinancialAccountant = pd.Series(df_courses_score['FinancialAccountant_Predict'], name='Predicted')

y_actu_FinancialAccountant = pd.Series(df_courses_score['FinancialAccountant_Label'], name='Actual')

df_confusion_FinancialAccountant = pd.crosstab(y_actu_FinancialAccountant, y_pred_FinancialAccountant , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [220]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: MACHINE LEARNING ENGINEER

y_actu_MachineLearning         = ''
y_pred_MachineLearning         = ''

df_courses_score['MachineLearning_Final_Score'] = (df_courses_score['MachineLearning_Role_Score'] * my_role_weight) + (df_courses_score['MachineLearning_Keyword_Score'] * my_skill_weight)

df_courses_score['MachineLearning_Predict'] = (df_courses_score['MachineLearning_Final_Score'] >= 0.5)

df_courses_score['MachineLearning_Label'] = df_courses_score.Category.isin(label_MachineLearning)

y_pred_MachineLearning = pd.Series(df_courses_score['MachineLearning_Predict'], name='Predicted')

y_actu_MachineLearning = pd.Series(df_courses_score['MachineLearning_Label'], name='Actual')

df_confusion_MachineLearning = pd.crosstab(y_actu_MachineLearning, y_pred_MachineLearning , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [221]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: MUSICIAN

y_actu_Musician         = ''
y_pred_Musician         = ''

df_courses_score['Musician_Final_Score'] = (df_courses_score['Musician_Role_Score'] * my_role_weight) + (df_courses_score['Musician_Keyword_Score'] * my_skill_weight)

df_courses_score['Musician_Predict'] = (df_courses_score['Musician_Final_Score'] >= 0.5)

df_courses_score['Musician_Label'] = df_courses_score.Category.isin(label_Musician)

y_pred_Musician = pd.Series(df_courses_score['Musician_Predict'], name='Predicted')

y_actu_Musician = pd.Series(df_courses_score['Musician_Label'], name='Actual')

df_confusion_Musician = pd.crosstab(y_actu_Musician, y_pred_Musician , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [222]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: NUTRITIONIST/DIETITIAN

y_actu_Dietitian         = ''
y_pred_Dietitian         = ''

df_courses_score['Dietitian_Final_Score'] = (df_courses_score['Dietitian_Role_Score'] * my_role_weight) + (df_courses_score['Dietitian_Keyword_Score'] * my_skill_weight)

df_courses_score['Dietitian_Predict'] = (df_courses_score['Dietitian_Final_Score'] >= 0.5)

df_courses_score['Dietitian_Label'] = df_courses_score.Category.isin(label_Dietitian)

y_pred_Dietitian = pd.Series(df_courses_score['Dietitian_Predict'], name='Predicted')

y_actu_Dietitian = pd.Series(df_courses_score['Dietitian_Label'], name='Actual')

df_confusion_Dietitian = pd.crosstab(y_actu_Dietitian, y_pred_Dietitian , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [223]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: Psychologist

y_actu_Psychologist         = ''
y_pred_Psychologist         = ''

df_courses_score['Psychologist_Final_Score'] = (df_courses_score['Psychologist_Role_Score'] * my_role_weight) + (df_courses_score['Psychologist_Keyword_Score'] * my_skill_weight)

df_courses_score['Psychologist_Predict'] = (df_courses_score['Psychologist_Final_Score'] >= 0.5)

df_courses_score['Psychologist_Label'] = df_courses_score.Category.isin(label_Psychologist)

y_pred_Psychologist = pd.Series(df_courses_score['Psychologist_Predict'], name='Predicted')

y_actu_Psychologist = pd.Series(df_courses_score['Psychologist_Label'], name='Actual')

df_confusion_Psychologist = pd.crosstab(y_actu_Psychologist, y_pred_Psychologist , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [224]:
df_confusion_DataScientist


Predicted,False,True
Actual,,
False,1420,661
True,2,130


In [225]:
df_confusion_SoftwareDevelopment

Predicted,False,True
Actual,,
False,1575,458
True,45,135


In [226]:
df_confusion_DatabaseAdministrator

Predicted,False,True
Actual,,
False,1456,746
True,1,10


In [227]:
df_confusion_Cybersecurity

Predicted,False,True
Actual,,
False,912,1271
True,4,26


In [228]:
df_confusion_FinancialAccountant

Predicted,False,True
Actual,,
False,810,1300
True,1,102


In [229]:
df_confusion_MachineLearning

Predicted,False,True
Actual,,
False,1009,1111
True,21,72


In [230]:
df_confusion_Musician

Predicted,False,True
Actual,,
False,1087,1089
True,18,19


In [231]:
df_confusion_Dietitian

Predicted,False,True
Actual,,
False,1001,1086
True,29,97


In [232]:
df_confusion_Psychologist

Predicted,False,True
Actual,,
False,1167,953
True,25,68


In [233]:
# Performance summary for the ROLE: DATA SCIENTIST


try:
    tn_DataScientist = df_confusion_DataScientist.iloc[0][False]
except:
    tn_DataScientist = 0
    
try:
    tp_DataScientist =  df_confusion_DataScientist.iloc[1][True]
except:
    tp_DataScientist = 0

    
try:
    fn_DataScientist = df_confusion_DataScientist.iloc[1][False]
except:
    fn_DataScientist = 0
    
try:
    fp_DataScientist =  df_confusion_DataScientist.iloc[0][True]
except:
    fp_DataScientist = 0  
    
    
total_count_DataScientist = tn_DataScientist + tp_DataScientist + fn_DataScientist + fp_DataScientist

print('Data Scientist Accuracy Rate : ', '{0:.2f}'.format((tn_DataScientist + tp_DataScientist) / total_count_DataScientist * 100))

print('Data Scientist Misclassifcation Rate : ',  '{0:.2f}'.format((fn_DataScientist + fp_DataScientist) / total_count_DataScientist * 100))

print('Data Scientist True Positive Rate : ',  '{0:.2f}'.format(tp_DataScientist / (tp_DataScientist + fn_DataScientist) * 100))

print('Data Scientist False Positive Rate : ',  '{0:.2f}'.format(fp_DataScientist / (tn_DataScientist + fp_DataScientist) * 100))


Data Scientist Accuracy Rate :  70.04
Data Scientist Misclassifcation Rate :  29.96
Data Scientist True Positive Rate :  98.48
Data Scientist False Positive Rate :  31.76


In [234]:
# Performance summary for the ROLE: SOFTWARE ENGINEER


try:
    tn_SoftwareDevelopment = df_confusion_SoftwareDevelopment.iloc[0][False]
except:
    tn_SoftwareDevelopment = 0
    
try:
    tp_SoftwareDevelopment =  df_confusion_SoftwareDevelopment.iloc[1][True]
except:
    tp_SoftwareDevelopment = 0

    
try:
    fn_SoftwareDevelopment = df_confusion_SoftwareDevelopment.iloc[1][False]
except:
    fn_SoftwareDevelopment = 0
    
try:
    fp_SoftwareDevelopment =  df_confusion_SoftwareDevelopment.iloc[0][True]
except:
    fp_SoftwareDevelopment = 0  
    
    
total_count_SoftwareDevelopment = tn_SoftwareDevelopment + tp_SoftwareDevelopment + fn_SoftwareDevelopment + fp_SoftwareDevelopment

print('Software Engineer Accuracy Rate : ', '{0:.2f}'.format((tn_SoftwareDevelopment + tp_SoftwareDevelopment) / total_count_SoftwareDevelopment * 100))

print('Software Engineer Misclassifcation Rate : ',  '{0:.2f}'.format((fn_SoftwareDevelopment + fp_SoftwareDevelopment) / total_count_SoftwareDevelopment * 100))

print('Software Engineer True Positive Rate : ',  '{0:.2f}'.format(tp_SoftwareDevelopment / (tp_SoftwareDevelopment + fn_SoftwareDevelopment) * 100))

print('Software Engineer False Positive Rate : ',  '{0:.2f}'.format(fp_SoftwareDevelopment / (tn_SoftwareDevelopment + fp_SoftwareDevelopment) * 100))


Software Engineer Accuracy Rate :  77.27
Software Engineer Misclassifcation Rate :  22.73
Software Engineer True Positive Rate :  75.00
Software Engineer False Positive Rate :  22.53


In [235]:
# Performance summary for the ROLE: DATABASE DEVELOPER/ ADMINISTRATOR


try:
    tn_DatabaseAdministrator = df_confusion_DatabaseAdministrator.iloc[0][False]
except:
    tn_DatabaseAdministrator = 0
    
try:
    tp_DatabaseAdministrator =  df_confusion_DatabaseAdministrator.iloc[1][True]
except:
    tp_DatabaseAdministrator = 0

    
try:
    fn_DatabaseAdministrator = df_confusion_DatabaseAdministrator.iloc[1][False]
except:
    fn_DatabaseAdministrator = 0
    
try:
    fp_DatabaseAdministrator =  df_confusion_DatabaseAdministrator.iloc[0][True]
except:
    fp_DatabaseAdministrator = 0  
    
    
total_count_DatabaseAdministrator = tn_DatabaseAdministrator + tp_DatabaseAdministrator + fn_DatabaseAdministrator + fp_DatabaseAdministrator

print('Database Administrator Accuracy Rate : ', '{0:.2f}'.format((tn_DatabaseAdministrator + tp_DatabaseAdministrator) / total_count_DatabaseAdministrator * 100))

print('Database Administrator Misclassifcation Rate : ',  '{0:.2f}'.format((fn_DatabaseAdministrator + fp_DatabaseAdministrator) / total_count_DatabaseAdministrator * 100))

print('Database Administrator True Positive Rate : ',  '{0:.2f}'.format(tp_DatabaseAdministrator / (tp_DatabaseAdministrator + fn_DatabaseAdministrator) * 100))

print('Database Administrator False Positive Rate : ',  '{0:.2f}'.format(fp_DatabaseAdministrator / (tn_DatabaseAdministrator + fp_DatabaseAdministrator) * 100))


Database Administrator Accuracy Rate :  66.24
Database Administrator Misclassifcation Rate :  33.76
Database Administrator True Positive Rate :  90.91
Database Administrator False Positive Rate :  33.88


In [236]:
# Performance summary for the ROLE: CYBERSECURITY CONSULTANT


try:
    tn_Cybersecurity = df_confusion_Cybersecurity.iloc[0][False]
except:
    tn_Cybersecurity = 0
    
try:
    tp_Cybersecurity =  df_confusion_Cybersecurity.iloc[1][True]
except:
    tp_Cybersecurity = 0

    
try:
    fn_Cybersecurity = df_confusion_Cybersecurity.iloc[1][False]
except:
    fn_Cybersecurity = 0
    
try:
    fp_Cybersecurity =  df_confusion_Cybersecurity.iloc[0][True]
except:
    fp_Cybersecurity = 0  
    
    
total_count_Cybersecurity = tn_Cybersecurity + tp_Cybersecurity + fn_Cybersecurity + fp_Cybersecurity

print('Cybersecurity Consultant Accuracy Rate : ', '{0:.2f}'.format((tn_Cybersecurity + tp_Cybersecurity) / total_count_Cybersecurity * 100))

print('Cybersecurity Consultant Misclassifcation Rate : ',  '{0:.2f}'.format((fn_Cybersecurity + fp_Cybersecurity) / total_count_Cybersecurity * 100))

print('Cybersecurity Consultant True Positive Rate : ',  '{0:.2f}'.format(tp_Cybersecurity / (tp_Cybersecurity + fn_Cybersecurity) * 100))

print('Cybersecurity Consultant False Positive Rate : ',  '{0:.2f}'.format(fp_Cybersecurity / (tn_Cybersecurity + fp_Cybersecurity) * 100))


Cybersecurity Consultant Accuracy Rate :  42.39
Cybersecurity Consultant Misclassifcation Rate :  57.61
Cybersecurity Consultant True Positive Rate :  86.67
Cybersecurity Consultant False Positive Rate :  58.22


In [237]:
# Performance summary for the ROLE: FINANCIAL ACCOUNTANT


try:
    tn_FinancialAccountant = df_confusion_FinancialAccountant.iloc[0][False]
except:
    tn_FinancialAccountant = 0
    
try:
    tp_FinancialAccountant =  df_confusion_FinancialAccountant.iloc[1][True]
except:
    tp_FinancialAccountant = 0

    
try:
    fn_FinancialAccountant = df_confusion_FinancialAccountant.iloc[1][False]
except:
    fn_FinancialAccountant = 0
    
try:
    fp_FinancialAccountant =  df_confusion_FinancialAccountant.iloc[0][True]
except:
    fp_FinancialAccountant = 0  
    
    
total_count_FinancialAccountant = tn_FinancialAccountant + tp_FinancialAccountant + fn_FinancialAccountant + fp_FinancialAccountant

print('Financial Accountant Consultant Accuracy Rate : ', '{0:.2f}'.format((tn_FinancialAccountant + tp_FinancialAccountant) / total_count_FinancialAccountant * 100))

print('Financial Accountant Consultant Misclassifcation Rate : ',  '{0:.2f}'.format((fn_FinancialAccountant + fp_FinancialAccountant) / total_count_FinancialAccountant * 100))

print('Financial Accountant Consultant True Positive Rate : ',  '{0:.2f}'.format(tp_FinancialAccountant / (tp_FinancialAccountant + fn_FinancialAccountant) * 100))

print('Financial Accountant Consultant False Positive Rate : ',  '{0:.2f}'.format(fp_FinancialAccountant / (tn_FinancialAccountant + fp_FinancialAccountant) * 100))


Financial Accountant Consultant Accuracy Rate :  41.21
Financial Accountant Consultant Misclassifcation Rate :  58.79
Financial Accountant Consultant True Positive Rate :  99.03
Financial Accountant Consultant False Positive Rate :  61.61


In [238]:
# Performance summary for the ROLE: MACHINE LEARNING ENGINEER


try:
    tn_MachineLearning = df_confusion_MachineLearning.iloc[0][False]
except:
    tn_MachineLearning = 0
    
try:
    tp_MachineLearning =  df_confusion_MachineLearning.iloc[1][True]
except:
    tp_MachineLearning = 0

    
try:
    fn_MachineLearning = df_confusion_MachineLearning.iloc[1][False]
except:
    fn_MachineLearning = 0
    
try:
    fp_MachineLearning =  df_confusion_MachineLearning.iloc[0][True]
except:
    fp_MachineLearning = 0  
    
    
total_count_MachineLearning = tn_MachineLearning + tp_MachineLearning + fn_MachineLearning + fp_MachineLearning

print('Machine Learning Engineer Accuracy Rate : ', '{0:.2f}'.format((tn_MachineLearning + tp_MachineLearning) / total_count_MachineLearning * 100))

print('Machine Learning Engineer Misclassifcation Rate : ',  '{0:.2f}'.format((fn_MachineLearning + fp_MachineLearning) / total_count_MachineLearning * 100))

print('Machine Learning Engineer True Positive Rate : ',  '{0:.2f}'.format(tp_MachineLearning / (tp_MachineLearning + fn_MachineLearning) * 100))

print('Machine Learning Engineer False Positive Rate : ',  '{0:.2f}'.format(fp_MachineLearning / (tn_MachineLearning + fp_MachineLearning) * 100))


Machine Learning Engineer Accuracy Rate :  48.85
Machine Learning Engineer Misclassifcation Rate :  51.15
Machine Learning Engineer True Positive Rate :  77.42
Machine Learning Engineer False Positive Rate :  52.41


In [239]:
# Performance summary for the ROLE: MUSICIAN


try:
    tn_Musician = df_confusion_Musician.iloc[0][False]
except:
    tn_Musician = 0
    
try:
    tp_Musician =  df_confusion_Musician.iloc[1][True]
except:
    tp_Musician = 0

    
try:
    fn_Musician = df_confusion_Musician.iloc[1][False]
except:
    fn_Musician = 0
    
try:
    fp_Musician =  df_confusion_Musician.iloc[0][True]
except:
    fp_Musician = 0  
    
    
total_count_Musician = tn_Musician + tp_Musician + fn_Musician + fp_Musician

print('Musician Accuracy Rate : ', '{0:.2f}'.format((tn_Musician + tp_Musician) / total_count_Musician * 100))

print('Musician Misclassifcation Rate : ',  '{0:.2f}'.format((fn_Musician + fp_Musician) / total_count_Musician * 100))

print('Musician True Positive Rate : ',  '{0:.2f}'.format(tp_Musician / (tp_Musician + fn_Musician) * 100))

print('Musician False Positive Rate : ',  '{0:.2f}'.format(fp_Musician / (tn_Musician + fp_Musician) * 100))


Musician Accuracy Rate :  49.98
Musician Misclassifcation Rate :  50.02
Musician True Positive Rate :  51.35
Musician False Positive Rate :  50.05


In [240]:
# Performance summary for the ROLE: DIETITIAN


try:
    tn_Dietitian = df_confusion_Dietitian.iloc[0][False]
except:
    tn_Dietitian = 0
    
try:
    tp_Dietitian =  df_confusion_Dietitian.iloc[1][True]
except:
    tp_Dietitian = 0

    
try:
    fn_Dietitian = df_confusion_Dietitian.iloc[1][False]
except:
    fn_Dietitian = 0
    
try:
    fp_Dietitian =  df_confusion_Dietitian.iloc[0][True]
except:
    fp_Dietitian = 0  
    
    
total_count_Dietitian = tn_Dietitian + tp_Dietitian + fn_Dietitian + fp_Dietitian

print('Dietitian Accuracy Rate : ', '{0:.2f}'.format((tn_Dietitian + tp_Dietitian) / total_count_Dietitian * 100))

print('Dietitian Misclassifcation Rate : ',  '{0:.2f}'.format((fn_Dietitian + fp_Dietitian) / total_count_Dietitian * 100))

print('Dietitian True Positive Rate : ',  '{0:.2f}'.format(tp_Dietitian / (tp_Dietitian + fn_Dietitian) * 100))

print('Dietitian False Positive Rate : ',  '{0:.2f}'.format(fp_Dietitian / (tn_Dietitian + fp_Dietitian) * 100))


Dietitian Accuracy Rate :  49.62
Dietitian Misclassifcation Rate :  50.38
Dietitian True Positive Rate :  76.98
Dietitian False Positive Rate :  52.04


In [241]:
# Performance summary for the ROLE: Psychologist


try:
    tn_Psychologist = df_confusion_Psychologist.iloc[0][False]
except:
    tn_Psychologist = 0
    
try:
    tp_Psychologist =  df_confusion_Psychologist.iloc[1][True]
except:
    tp_Psychologist = 0

    
try:
    fn_Psychologist = df_confusion_Psychologist.iloc[1][False]
except:
    fn_Psychologist = 0
    
try:
    fp_Psychologist =  df_confusion_Psychologist.iloc[0][True]
except:
    fp_Psychologist = 0  
    
    
total_count_Psychologist = tn_Psychologist + tp_Psychologist + fn_Psychologist + fp_Psychologist

print('Psychologist Accuracy Rate : ', '{0:.2f}'.format((tn_Psychologist + tp_Psychologist) / total_count_Psychologist * 100))

print('Psychologist Misclassifcation Rate : ',  '{0:.2f}'.format((fn_Psychologist + fp_Psychologist) / total_count_Psychologist * 100))

print('Psychologist True Positive Rate : ',  '{0:.2f}'.format(tp_Psychologist / (tp_Psychologist + fn_Psychologist) * 100))

print('Psychologist False Positive Rate : ',  '{0:.2f}'.format(fp_Psychologist / (tn_Psychologist + fp_Psychologist) * 100))


Psychologist Accuracy Rate :  55.81
Psychologist Misclassifcation Rate :  44.19
Psychologist True Positive Rate :  73.12
Psychologist False Positive Rate :  44.95


### End of the Notebook. Thank you!